# 조건 & 이동평균

In [1]:
import FinanceDataReader as fdr
import numpy as np

df = fdr.DataReader('006650', start = '2021')

df = df[['Close']].copy()
df

,Close
Date,
2021-01-04,234000
2021-01-05,235000
2021-01-06,277000
2021-01-07,290500
2021-01-08,285000
...,...
2022-07-11,126000
2022-07-12,117500
2022-07-13,122500


In [2]:
# np.where 사용

df['Change'] = df['Close'] - df['Close'].shift(1)
df

,Close,Change
Date,,
2021-01-04,234000,NaN
2021-01-05,235000,1000.0
2021-01-06,277000,42000.0
2021-01-07,290500,13500.0
2021-01-08,285000,-5500.0
...,...,...
2022-07-11,126000,-2500.0
2022-07-12,117500,-8500.0
2022-07-13,122500,5000.0


In [3]:
df['Gain'] = np.where(df['Change']>=0, df['Change'], 0)
df['Loss'] = np.where(df['Change']<0, df['Change']*-1, 0)
df

,Close,Change,Gain,Loss
Date,,,,
2021-01-04,234000,NaN,0.0,0.0
2021-01-05,235000,1000.0,1000.0,0.0
2021-01-06,277000,42000.0,42000.0,0.0
2021-01-07,290500,13500.0,13500.0,0.0
2021-01-08,285000,-5500.0,0.0,5500.0
...,...,...,...,...
2022-07-11,126000,-2500.0,0.0,2500.0
2022-07-12,117500,-8500.0,0.0,8500.0
2022-07-13,122500,5000.0,5000.0,0.0


In [4]:
df['Gain'].rolling(window = 14).mean()

Date
2021-01-04            NaN
2021-01-05            NaN
2021-01-06            NaN
2021-01-07            NaN
2021-01-08            NaN
                 ...     
2022-07-11    1071.428571
2022-07-12    1071.428571
2022-07-13    1428.571429
2022-07-14    1178.571429
2022-07-15     964.285714
Name: Gain, Length: 380, dtype: float64

In [5]:
# 이동평균
df['avgGain'] = df['Gain'].rolling(window = 14).mean()
df['avgLoss'] = df['Loss'].rolling(window = 14).mean()
df

df.head(20)

,Close,Change,Gain,Loss,avgGain,avgLoss
Date,,,,,,
2021-01-04,234000,NaN,0.0,0.0,NaN,NaN
2021-01-05,235000,1000.0,1000.0,0.0,NaN,NaN
2021-01-06,277000,42000.0,42000.0,0.0,NaN,NaN
2021-01-07,290500,13500.0,13500.0,0.0,NaN,NaN
2021-01-08,285000,-5500.0,0.0,5500.0,NaN,NaN
2021-01-11,277000,-8000.0,0.0,8000.0,NaN,NaN
2021-01-12,282000,5000.0,5000.0,0.0,NaN,NaN
2021-01-13,277500,-4500.0,0.0,4500.0,NaN,NaN
2021-01-14,284000,6500.0,6500.0,0.0,NaN,NaN


# RSI 함수 만들기

In [6]:
import FinanceDataReader as fdr
import numpy as np

df = fdr.DataReader('006650', start = '2021')

df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,233000,240000,231000,234000,119512,0.017391
2021-01-05,234000,236500,227500,235000,78425,0.004274
2021-01-06,255000,287500,249500,277000,621523,0.178723
2021-01-07,291000,298500,278000,290500,251334,0.048736
2021-01-08,295000,295500,280000,285000,174917,-0.018933
...,...,...,...,...,...,...
2022-07-11,130500,130500,125500,126000,34005,-0.019455
2022-07-12,125500,126500,116000,117500,68785,-0.067460
2022-07-13,118500,123000,118500,122500,36296,0.042553


In [7]:
def rsi_fn(df, rsi_period):
    
    df_close = df[['Close']].copy()
    
    df_close['Change'] = df_close['Close'] - df_close['Close'].shift(1)
    
    df_close['Gain'] = np.where(df_close['Change']>=0, df_close['Change'], 0)
    
    df_close['Loss'] = np.where(df_close['Change']<0, df_close['Change']*-1, 0)
    
    df_close['avgGain'] = df_close['Gain'].rolling(window = rsi_period).mean()
    
    df_close['avgLoss'] = df_close['Loss'].rolling(window = rsi_period).mean()
    
    df_close['RS'] = df_close["avgGain"] / df_close['avgLoss']
    
    df_close['RSI'] = 100 - (100/(1+df_close['RS']))
    
    return df_close

In [8]:
df_rsi = rsi_fn(df, rsi_period = 14)
df_rsi.head(20)

,Close,Change,Gain,Loss,avgGain,avgLoss,RS,RSI
Date,,,,,,,,
2021-01-04,234000,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2021-01-05,235000,1000.0,1000.0,0.0,NaN,NaN,NaN,NaN
2021-01-06,277000,42000.0,42000.0,0.0,NaN,NaN,NaN,NaN
2021-01-07,290500,13500.0,13500.0,0.0,NaN,NaN,NaN,NaN
2021-01-08,285000,-5500.0,0.0,5500.0,NaN,NaN,NaN,NaN
2021-01-11,277000,-8000.0,0.0,8000.0,NaN,NaN,NaN,NaN
2021-01-12,282000,5000.0,5000.0,0.0,NaN,NaN,NaN,NaN
2021-01-13,277500,-4500.0,0.0,4500.0,NaN,NaN,NaN,NaN
2021-01-14,284000,6500.0,6500.0,0.0,NaN,NaN,NaN,NaN
